## 0. Configurações e Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuração do diretório de saída para os gráficos
# OUTPUT_DIR = "analytical_report"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

## 1. Extração e Processamento dos Dados

In [2]:
def extract_data():
    """Extrai e processa os dados de metais do site westmetall."""
    print("Iniciando extração de dados...")
    
    METALS_URLS = {
        'copper': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash',
        'tin': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Sn_cash',
        'lead': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Pb_cash',
        'zinc': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Zn_cash',
        'aluminium': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash',
        'nickel': 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Ni_cash',
    }

    raw_dfs = {}
    for metal, url in METALS_URLS.items():
        print(f"Extraindo dados de {metal.capitalize()}...")
        try:
            table = pd.read_html(url)
            df = pd.concat(table, ignore_index=True)
            df['metal'] = metal
            raw_dfs[metal] = df
        except Exception as e:
            print(f"Falha ao extrair dados para {metal}: {e}")
            continue
    
    print("Extração de dados concluída.\n")
    return raw_dfs

def standardize_and_clean(raw_dfs):
    """Padroniza e limpa os dataframes."""
    print("Iniciando padronização e limpeza dos dados...")

    METALS = list(raw_dfs.keys())
    COLUMN_MAP = {}
    for metal in METALS:
        metal_cap = metal.capitalize()
        COLUMN_MAP.update({
            f"LME {metal_cap} Cash-Settlement": "cash",
            f"LME {metal_cap} 3-month": "three_month",
            f"LME {metal_cap} stock": "stock",
        })

    def standardize_df(df, metal):
        df = df.rename(columns=COLUMN_MAP)
        df["metal"] = metal
        df = df[["metal", "date", "cash", "three_month", "stock"]]
        return df

    def fix_date_column(df, col="date"):
        if col not in df.columns: return df
        df = df[df[col].str.contains(r"\d", regex=True, na=False)].copy()
        df[col] = pd.to_datetime(df[col].str.replace(".", "", regex=False))
        return df

    def convert_numeric_columns(df):
        cols = ["cash", "three_month", "stock"]
        for col in cols:
            if col in df.columns:
                df[col] = (
                    df[col]
                    .astype(str)
                    .str.replace(".", "", regex=False)
                    .str.replace(",", ".", regex=False)
                    .str.replace(" ", "", regex=False)
                )
                df[col] = pd.to_numeric(df[col], errors="coerce")
        return df

    dfs_metals = {metal: standardize_df(df, metal) for metal, df in raw_dfs.items()}
    
    dfs_metals_fixed = {
        metal: fix_date_column(df).pipe(convert_numeric_columns)
        for metal, df in dfs_metals.items()
    }
    
    print("Limpeza de dados concluída.\n")
    return dfs_metals_fixed

In [3]:
raw_data = extract_data()
cleaned_data = standardize_and_clean(raw_data)

Iniciando extração de dados...
Extraindo dados de Copper...
Extraindo dados de Tin...
Extraindo dados de Lead...
Extraindo dados de Zinc...
Extraindo dados de Aluminium...
Extraindo dados de Nickel...
Extração de dados concluída.

Iniciando padronização e limpeza dos dados...
Limpeza de dados concluída.



## 2. Criação de Features

In [4]:
def create_features(dfs_metals_fixed):
    """Cria features analíticas para cada metal."""
    print("Iniciando criação de features...")
    
    dfs_features = {}
    for metal, df_orig in dfs_metals_fixed.items():
        df = df_orig.copy()
        df = df.sort_values("date", ascending=True).reset_index(drop=True)

        # Features
        df["cash_return"] = df["cash"].pct_change(fill_method=None) # variação percentual diária do cash por metal
        df["vol_30d"] = df["cash_return"].rolling(window=30).std() # volatilidade (desvio-padrão) dos retornos em uma janela de 30 dias
        df["basis"] = df["three_month"] - df["cash"] # (contango/backwardation)
        df["stock_change"] = df["stock"].diff() # variação diária de estoque
        df['year'] = df['date'].dt.year  # ano da data

        dfs_features[metal] = df
        
    print("Criação de features concluída.\n")
    return dfs_features

In [5]:
featured_data = create_features(cleaned_data)

Iniciando criação de features...
Criação de features concluída.



In [6]:
featured_data['copper']

,metal,date,cash,three_month,stock,cash_return,vol_30d,basis,stock_change,year
0,copper,2008-01-02,666600.0,670600.0,198925.0,NaN,NaN,4000.0,NaN,2008
1,copper,2008-01-03,676500.0,681800.0,198175.0,0.014851,NaN,5300.0,-750.0,2008
2,copper,2008-01-04,699050.0,703500.0,198600.0,0.033333,NaN,4450.0,425.0,2008
3,copper,2008-01-07,691550.0,695050.0,201000.0,-0.010729,NaN,3500.0,2400.0,2008
4,copper,2008-01-08,710600.0,714050.0,200975.0,0.027547,NaN,3450.0,-25.0,2008
...,...,...,...,...,...,...,...,...,...,...
4521,copper,2025-11-17,1079900.0,1080300.0,136050.0,-0.004792,0.010223,400.0,325.0,2025
4522,copper,2025-11-18,1064900.0,1069500.0,140500.0,-0.013890,0.010547,4600.0,4450.0,2025
4523,copper,2025-11-19,1078400.0,1081500.0,157875.0,0.012677,0.010677,3100.0,17375.0,2025
4524,copper,2025-11-20,1078900.0,1080300.0,157925.0,0.000464,0.010444,1400.0,50.0,2025


In [7]:
# Salva os dataframes processados em arquivos CSV
output_dir = "processed_data"
os.makedirs(output_dir, exist_ok=True)
for metal, df in featured_data.items():
    output_path = os.path.join(output_dir, f"{metal}.csv")
    df.to_csv(output_path, index=False)
    print(f"Dados processados de {metal} salvos em {output_path}.")

Dados processados de copper salvos em processed_data/copper.csv.
Dados processados de tin salvos em processed_data/tin.csv.
Dados processados de lead salvos em processed_data/lead.csv.
Dados processados de zinc salvos em processed_data/zinc.csv.
Dados processados de aluminium salvos em processed_data/aluminium.csv.
Dados processados de nickel salvos em processed_data/nickel.csv.
